In [ ]:
import json
import time
from pathlib import Path
import argparse

import pandas as pd
import plotly.express as px


input_path = Path("/input/projects/dhs/smartsensingandimaging/development/fops")
output_path = Path("/output/projects/dhs/smartsensingandimaging/fops")
days = ["2022-12-06"]
cameras = "*220700207"

# cameras = "*"
# days = ["2022-08-31", "2022-09-02"]
# days = ["2022-07-22", "2022-08-03"]

log_ = []
def log(var):
    log_.append((time.time(), var))

In [ ]:
def get_serial(name):
    return name.split("_")[-1] if "_" in name else name

def get_transform(calibration, camera):
    return calibration[get_serial(camera)]["transform"]

def read_calibration(calibration_A, calibration_B):
    calibration = {}
    for head, path in {"A": calibration_A, "B": calibration_B}.items():
        with open(path, "r") as fp:
            cameras = json.load(fp)["Calibration"]["cameras"]
            for camera in cameras:
                camera["head"] = head
                calibration[camera["serial"]] = camera
    return calibration

def get_side(config, path, calibration):
    if config["method"] == "by_head":
        identifier = calibration.get(get_serial(Path(path).parts[-2]))["head"]
    elif config["method"] == "by_run":
        identifier = int(Path(path).parts[-3])
    if not isinstance(config["north"], list):
        config["north"] = [config["north"]]
    return "north" if identifier in config["north"] else "south"

# 
x_offset_map = {"north": 0, "south": -1}

def plot_locs(df):
    fig = px.scatter(df, x="x", y="y", color="pass", facet_col="side")
    fig.update_yaxes(scaleanchor = "x", scaleratio = 1)
    fig.update_layout(width=400 * len(df["side"].unique()) + 100, height=500)
    # fig.update_layout(title=f"Location plots for {', '.join([d.name for d in df['day'].unique()])}")
    fig.update_traces(marker={'size': 6})
    return fig

def extract_metadata(days, calibration, side_cfg, cameras="*"):
    if not isinstance(days, list):
        days = [days]

    metadata = []
    for day in days:
        for run in Path(day).glob("*"):
            # Ignore files
            if not run.is_dir():
                continue
            for camera in run.glob(cameras):
                # Ignore files
                if not camera.is_dir():
                    continue
                if calibration.get(get_serial(camera.name)) is None:
                    print(f"Warning: camera {camera} not found in calibration file, skipping.")
                    continue
                for meta_f in camera.glob("*.json"):
                    # Ignore empty json files
                    if meta_f.stat().st_size > 0:
                        day_, run, camera, js = meta_f.parts[-4:]
                        meta = {"image": meta_f.with_suffix(".png").name, "day": day, "side": get_side(side_cfg, meta_f, calibration), "camera": camera, "run": run}
                        with open(meta_f, "r") as fp:
                            meta.update(json.load(fp))
                        metadata.append(meta)

    metadata = pd.DataFrame(metadata)
    transform = pd.json_normalize([get_transform(calibration, row["camera"]) for _, row in metadata.iterrows()])
    metadata["x"] += transform["translation.x"] * metadata["side"].map({"north": 1, "south": -1}) * 1000
    metadata["y"] += transform["translation.y"] * 1000
#     metadata["z"] = (transform["translation.z"] + metadata["side"].map({"north": 0, "south": -1.6})) * 1000
#     metadata["rot_x"] = transform["rotation.rx"]
#     metadata["rot_y"] = transform["rotation.ry"] + metadata["side"].map({"north": 0, "south": np.pi})
#     metadata["rot_z"] = transform["rotation.rz"]
    
    # Assign a pass number for locations that have been visited multiple times in one scan
    metadata.sort_values(["side", "camera", "y", "x", "milliseconds_system"], inplace=True)
    metadata["pass"] = metadata.groupby(["side", "camera", "y", "x"]).cumcount() + 1
    last_pass = metadata.groupby(["side", "camera", "y", "x"], as_index=False).last()

    return metadata, last_pass


In [ ]:
def save_scan(days, cameras="*"):
    if not isinstance(days, list):
        days = [days]

    cfg_path = days[0].joinpath("config.json")
    if not cfg_path.is_file():
        print(f"Warning: {cfg_path} not found, skipping.")
        return None, None
    with open(cfg_path, "r") as fp:
        config = json.load(fp)
    calibration = read_calibration(config["calibration_A"], config["calibration_B"])

    metadata, last_pass = extract_metadata(days, calibration, config["side_assignment"], cameras)
    metadata.set_index("image").to_csv(days[0].joinpath("metadata_full.csv"))
    last_pass.set_index("image").to_csv(days[0].joinpath("metadata_last.csv"))
    print(f"CSVs written to {str(days[0])}")
    return metadata, last_pass

In [ ]:
days = [input_path.joinpath(day) for day in days]
metadata, last_pass = save_scan(days, cameras)
fig = plot_locs(last_pass)
fig.show()
# fig.write_html("locations.html")

# pd.DataFrame({"Last-pass image counts": last_pass.value_counts(["camera", "side"])})

In [ ]:
fig.update_traces(marker={'size': 6})

In [ ]:
# multi_day = [["2022-08-31", "2022-09-02"], ["2022-07-22", "2022-08-03"], ]
# multi_day = [[input_path.joinpath(day) for day in scan] for scan in multi_day]
# scans = input_path.glob("*")
# scans = [scan for scan in scans if scan.is_dir() and not scan in [d for scan in multi_day for d in scan]]
# scans = sorted(scans)
# scans += multi_day

# counts = []
# for scan in scans:
#     print(f"Processing {scan}.")
#     metadata, last_pass = save_scan(scan, plot=True)
#     if last_pass is not None:
#         counts.append(last_pass.value_counts(["camera", "side"]))